In [1]:
import torch
print('Version', torch.__version__)
print('CUDA enabled:', torch.cuda.is_available())

Version 1.7.0+cu101
CUDA enabled: True


In [2]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [3]:
import os
BASE_PATH = '/gdrive/My Drive/colab_files/Final Project/'
os.chdir(BASE_PATH)

In [4]:
!pip install gym[box2d]

In [5]:
!pip3 install -r requirements.txt

In [6]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

In [7]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay

In [8]:
from utils import *
from models import VAE, MDRNNCell, Controller

In [9]:
import pyglet
print(pyglet.version)
import argparse
import sys
from os.path import join, exists, abspath
from os import mkdir, unlink, listdir, getpid
from time import sleep
from torch.multiprocessing import Process, Queue
import torch
import cma
from models import Controller
from tqdm import tqdm
import numpy as np
from utils.misc import RolloutGenerator, ASIZE, RSIZE, LSIZE
from utils.misc import load_parameters
from utils.misc import flatten_parameters
from torch.multiprocessing import Process, Queue
import multiprocessing

1.5.0


In [10]:
ASIZE, LSIZE, RSIZE, RED_SIZE, SIZE =\
    3, 32, 64, 64, 64

In [11]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

# This code creates a virtual display to draw game images on. 
# If you are running locally, just ignore it
import os
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY"))==0:
    !bash ../xvfb start
    %env DISPLAY=:1

In [12]:
time_limit = 1000
device = "cuda"
pop_size = 4
n_samples = 4
target_return = 950
display = True
controller = Controller(LSIZE, RSIZE, ASIZE)
parameters = controller.parameters()
es = cma.CMAEvolutionStrategy(flatten_parameters(parameters), 0.1,
                              {'popsize': pop_size})
vae_weights_path = "weights/vae_original.pt"
mdrnn_weights_path = "weights/mdrnn.pt"
rg = RolloutGenerator("weights/vae_original.pt", "weights/mdrnn.pt", device, time_limit)

(2_w,4mirr1)-aCMA-ES (mu_w=1.5,w_1=80%) in dimension 291 (seed=876978, Sun Dec 13 01:59:41 2020)


In [13]:
from collections import deque
p_queue = deque()
r_queue = deque()

In [14]:
def evaluate(solutions, results, rollouts=10):
    """ Give current controller evaluation.
    Evaluation is minus the cumulated reward averaged over rollout runs.
    :args solutions: CMA set of solutions
    :args results: corresponding results
    :args rollouts: number of rollouts
    :returns: minus averaged cumulated reward
    """
    index_min = np.argmin(results)
    best_guess = solutions[index_min]
    restimates = []

    for s_id in range(rollouts):
        p_queue.append((s_id, best_guess))
    
    while  len(p_queue) != 0:
      id, best_guess = p_queue.popleft()
      r_queue.append((id, rg.rollout(best_guess)))

    print("Evaluating...")
    for _ in tqdm(range(rollouts)):
        restimates.append(r_queue.popleft()[1])

    return best_guess, np.mean(restimates), np.std(restimates)

In [15]:
pop_size * n_samples

16

In [ ]:
epoch = 0
log_step = 3
while epoch < 3:
    print(f"Epoch: {epoch}")
    if cur_best is not None and - cur_best > target_return:
        print("Already better than target, breaking...")
        break

    r_list = [0] * pop_size  # result list
    solutions = es.ask()
    print(f"Number of solutions: {len(solutions)}")
    # push parameters to queue
    for s_id, s in enumerate(solutions):
        for _ in range(n_samples):
            p_queue.append((s_id, s))
    print(f"Filled up p_queue of size {len(p_queue)}")
    count = 0
    while len(p_queue) != 0:
      if count % pop_size == 0:
        print(count)
      id, sol = p_queue.popleft()
      r_queue.append((id, rg.rollout(sol)))
      count += 1
    print(f"Filled up r_queue of size {len(r_queue)}")
    # retrieve results
    if display:
        pbar = tqdm(total=pop_size * n_samples)
    for _ in range(pop_size * n_samples):
        r_s_id, r = r_queue.popleft()
        r_list[r_s_id] += r / n_samples
        if display:
            pbar.update(1)
    print("Filled up r_list")
    if display:
        pbar.close()

    es.tell(solutions, r_list)
    es.disp()

    # evaluation and saving
    if epoch % log_step == log_step - 1:
        best_params, best, std_best = evaluate(solutions, r_list)
        print("Current evaluation: {}".format(best))
        if not cur_best or cur_best > best:
            cur_best = best
            print(f"New best is {cur_best}")
            # print("Saving new best with value {}+-{}...".format(-cur_best, std_best))
            # load_parameters(best_params, controller)
            # torch.save(
            #     {'epoch': epoch,
            #      'reward': - cur_best,
            #      'state_dict': controller.state_dict()},
            #     join(ctrl_dir, 'best.tar'))
        if - best > target_return:
            print("Terminating controller training with value {}...".format(best))
            break
    epoch += 1
    torch.cuda.empty_cache()


Epoch: 0
Number of solutions: 4
Filled up p_queue of size 16
0
Track generation: 1199..1503 -> 304-tiles track
Track generation: 1026..1287 -> 261-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1196..1499 -> 303-tiles track
Track generation: 1406..1762 -> 356-tiles track
Track generation: 1361..1705 -> 344-tiles track
4
Track generation: 904..1138 -> 234-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1252..1569 -> 317-tiles track
Track generation: 1191..1493 -> 302-tiles track
Track generation: 1112..1394 -> 282-tiles track
Track generation: 1135..1423 -> 288-tiles track
8
Track generation: 1272..1593 -> 321-tiles track
Track generation: 1119..1403 -> 284-tiles track
Track generation: 1072..1344 -> 272-tiles track
Track generation: 1153..1445 -> 292-tiles track
12
Track generation: 1176..1474 -> 298-tiles track
Track generation: 1361..1705 -> 344-tiles track
Tra

100%|██████████| 16/16 [00:00<00:00, 37680.44it/s]

Filled up r_queue of size 16
Filled up r_list
    4     16 8.276871009774811e+01 1.0e+00 9.48e-02  9e-02  9e-02 7:08.8
Epoch: 1
Number of solutions: 4
Filled up p_queue of size 16
0
Track generation: 1190..1491 -> 301-tiles track


Track generation: 1188..1489 -> 301-tiles track
Track generation: 1065..1344 -> 279-tiles track
Track generation: 1160..1464 -> 304-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 906..1143 -> 237-tiles track
4
Track generation: 1186..1488 -> 302-tiles track
Track generation: 1185..1486 -> 301-tiles track
Track generation: 1059..1328 -> 269-tiles track
Track generation: 1013..1274 -> 261-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1273..1595 -> 322-tiles track
8
Track generation: 906..1143 -> 237-tiles track
Track generation: 1071..1343 -> 272-tiles track
Track generation: 950..1197 -> 247-tiles track
Track generation: 1089..1365 -> 276-tiles track
12
Track generation: 1034..1304 -> 270-tiles track
Track generation: 1212..1526 -> 314-tiles track
Track generation: 1205..1510 -> 305-tiles track


In [1]:
!nvidia-smi

Sun Dec 13 02:10:41 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    23W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------